<a href="https://colab.research.google.com/github/JSunkel/Forecasting-Models/blob/master/Marketing%20Campaign%20Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

In [0]:
# Define a function to calculate the results of a marketing campaign
def run_campaign(spend, cpm, conversion_rate):
  return np.random.binomial(spend / cpm * 1000, conversion_rate)

In [0]:
# From the expected conversion rate and standard deviation, get the realized conversion rate
def get_conversion_rate(expected, stdev):
  conversion_rate = max(expected + np.random.normal() * stdev, 0.000001)
  return conversion_rate

In [0]:
# Budget
spend = 50000

In [0]:
# Cost per thousand
cpm = 2

In [0]:
# Conversion rate
conversion_rate_expected = 0.00005
conversion_rate_stdev = 0.00002

In [83]:
# Estimate the number of new customers from the campaign
conversion_rate = get_conversion_rate(conversion_rate_expected,
                                      conversion_rate_stdev)
cohort_size = run_campaign(spend, cpm, conversion_rate)

# Calculate Customer Acquiaition Cost
CAC = spend / cohort_size

print('Customers gained: ', cohort_size)
print('CAC (Customer Acquisition Cost): ', round(CAC, 2))

Customers gained:  1666
CAC (Customer Acquisition Cost):  30.01


In [0]:
# Define a function that models the progression of a cohort over time
def simulate_cohort(cohort_size, 
                    churn_rate, 
                    transactions, 
                    price, 
                    retention_cost, 
                    yrs = 5):
  customers_left = []
  spending = []
  profit = []
  for i in range(yrs):
    for customer in range(cohort_size):
      # For simplicity, assume cancels happen at the start of the year
      churn_random_num = np.random.random()
      # Generate a random number between 0 and 1. If less than churn_rate then
      # the customer churned and we subtract 1 from cohort_size
      if churn_random_num <= churn_rate:
        cohort_size += -1
      # Calculate and record cohort's data
      customers_left.append(cohort_size)
      spending.append(cohort_size * transactions * price)
      profit.append(cohort_size * (transactions * price - retention_cost))
  return customers_left, spending, profit

In [0]:
churn_rate = 0.20

In [0]:
# NUmber of annual transactions per average cohort member
transactions = 6

In [0]:
# Price of goods sold per average transaction
price = 10

In [0]:
# Annual cost of retaining/servicing customer
retention_cost = 20

In [0]:
# Run the function
customers_left, spending, profit = simulate_cohort(cohort_size, 
                                                   churn_rate, 
                                                   transactions, 
                                                   price, 
                                                   retention_cost, 
                                                   yrs = 5)

In [0]:
# Define a function to calculate present value
def present_value(cashflows, rate):
  pvs = []
  for i, val in enumerate(cashflows):
    pvs.append(val / (1 + rate) ** (i + 1))
  return pvs

In [91]:
# Calculate CLTV
rate = 0.10

# Get the PV of the profits
pvs = present_value(profit, rate)

# Value of the cohort in today's dollars is the sum of the PVs
cohort_value = sum(pvs)

print('Total Cohort Value: ', int(cohort_value))
print('CLTV: ', int(cohort_value / cohort_size))
print('CLTV - CAC Spread: ', int(cohort_value / cohort_size - CAC))

Total Cohort Value:  665795
CLTV:  399
CLTV - CAC Spread:  369
